앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.


이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").


항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.


메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

In [2]:
import sys
import os
sys.path.append('../..')
from get_env import get_env

api_key = get_env()


In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

# 모델 초기화
llm = ChatOpenAI(temperature=0.1)

# 메모리 초기화
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

# 영화 이모티콘 예시들
examples = [
    {
        "movie": "탑건",
        "emojis": "🛩️👨‍✈️🔥"
    },
    {
        "movie": "대부",
        "emojis": "👨‍👨‍👦🔫🍝"
    },
    {
        "movie": "타이타닉",
        "emojis": "🚢💔🌊"
    },
    {
        "movie": "어벤져스",
        "emojis": "🦸‍♂️⚡🛡️"
    },
    {
        "movie": "라이온킹",
        "emojis": "🦁👑🌅"
    }
]

# FewShotChatMessagePromptTemplate 설정
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{movie}"),
    ("ai", "{emojis}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 템플릿
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 제목을 받아서 그 영화를 나타내는 세 개의 이모티콘으로 응답하는 AI입니다. 항상 정확히 세 개의 이모티콘만 반환하세요."),
    few_shot_prompt,
    ("human", "{movie}")
])

print("설정 완료!")


설정 완료!


In [4]:
# 메모리와 함께 사용할 프롬프트 템플릿
memory_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 제목을 받아서 그 영화를 나타내는 세 개의 이모티콘으로 응답하는 AI입니다. 항상 정확히 세 개의 이모티콘만 반환하세요."),
    few_shot_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{movie}")
])

# 메모리 로드 함수
def load_memory(_):
    return memory.load_memory_variables({})["history"]

# LCEL 체인 구성
chain = RunnablePassthrough.assign(history=load_memory) | memory_prompt | llm

# 체인 실행 함수
def invoke_movie_chain(movie_title):
    result = chain.invoke({"movie": movie_title})
    memory.save_context(
        {"input": movie_title},
        {"output": result.content},
    )
    print(f"영화: {movie_title}")
    print(f"이모티콘: {result.content}")
    print("-" * 30)
    return result.content

print("LCEL 체인 구성 완료!")


LCEL 체인 구성 완료!


In [5]:
# 첫 번째 영화 질문
invoke_movie_chain("인터스텔라")

영화: 인터스텔라
이모티콘: 🚀⏳🌌
------------------------------


'🚀⏳🌌'

In [6]:
# 두 번째 영화 질문
invoke_movie_chain("해리포터")

영화: 해리포터
이모티콘: ⚡🧙‍♂️🔮
------------------------------


'⚡🧙\u200d♂️🔮'

In [7]:
# 메모리 작동 확인 - 이전에 질문한 영화가 무엇인지 물어보기
def ask_about_previous_movies():
    question_prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 사용자가 이전에 질문한 영화들을 기억하고 있습니다. 사용자가 질문한 영화들을 알려주세요."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "내가 이전에 질문한 영화들이 뭐였지?")
    ])
    
    question_chain = RunnablePassthrough.assign(history=load_memory) | question_prompt | llm
    result = question_chain.invoke({})
    print("이전 질문 확인:")
    print(result.content)
    return result.content

ask_about_previous_movies()


이전 질문 확인:
당신이 이전에 질문한 영화는 '인터스텔라'와 '해리포터'입니다.


"당신이 이전에 질문한 영화는 '인터스텔라'와 '해리포터'입니다."